"""
Contexto:
En este ejercicio, trabajaremos en un problema de logística para optimizar el proceso de exportación de vinos desde las reconocidas regiones vitivinícolas de Chile (Maipo, Casablanca, Colchagua) hacia los puertos de Valparaíso y San Antonio.

Objetivo:
El objetivo es minimizar el costo total del transporte del vino, considerando las capacidades de producción de cada región y la demanda de los puertos para exportación.

Datos del Problema:
- Oferta (Capacidad de Producción en pallets):
  - Maipo: 240 pallets
  - Casablanca: 360 pallets
  - Colchagua: 500 pallets
- Demanda (Requerimientos de los Puertos en pallets):
  - Valparaíso: 475 pallets
  - San Antonio: 510 pallets
- Costos de Transporte (CLP por pallet):
  - Maipo a Valparaíso: 22,000
  - Maipo a San Antonio: 26,000
  - Casablanca a Valparaíso: 18,000
  - Casablanca a San Antonio: 23,000
  - Colchagua a Valparaíso: 30,000
  - Colchagua a San Antonio: 28,000

Tareas:
1. Modelar el Problema: Utiliza Pyomo para definir el modelo de programación lineal. Incluye variables de decisión, función objetivo y restricciones.
2. Variables de Decisión: Cantidad de pallets de vino a transportar desde cada región a cada puerto.
3. Función Objetivo: Minimizar el costo total de transporte.
4. Restricciones:
   - Respetar la capacidad de producción de cada región vitivinícola.
   - Satisfacer la demanda de cada puerto.
5. Resolver el Modelo: Usa un solver como GLPK para encontrar la solución óptima.
6. Análisis de Resultados: Interpreta los resultados para entender la distribución óptima de envíos y el costo total del transporte.

Notas Adicionales:
- Este ejercicio es una oportunidad para aplicar y comprender la programación lineal en un contexto real y relevante.
- Considera las implicaciones de la solución obtenida en términos de logística y eficiencia operativa.
"""

In [ ]:
# Importamos la biblioteca pyomo

In [128]:
from pyomo.environ import *

In [ ]:
# Datos del problema

In [129]:
Oferta = {'Maipo': 240, 'Casablanca': 360, 'Colchagua': 500} # Pallets disponibles en cada viñedo

In [131]:
Demanda = {'Valparaiso': 475, 'SanAntonio': 510} # Pallets requerido en cada Puerto con ruta comercial hacia el mercado objetivo

In [130]:
Costos = {('Maipo', 'Valparaiso'): 22000, ('Maipo', 'SanAntonio'): 26000,
          ('Casablanca', 'Valparaiso'): 18000, ('Casablanca', 'SanAntonio'): 23000,
          ('Colchagua', 'Valparaiso'): 30000, ('Colchagua', 'SanAntonio'): 28000} # Costos por pallet

In [ ]:
# Crear el modelo de Pyomo

In [132]:
modelo = ConcreteModel()

In [ ]:
# Crea las lista VINEDOS y PUERTOS que contiene los nombres de todas las regiones vitivinícolas y puertos
# definidas en los diccionario Ofertas y Demanda respectivamente

In [133]:
VINEDOS = list(Oferta.keys())

In [134]:
PUERTOS = list(Demanda.keys())

In [135]:
# Definimos las Variables de decisión
# x es la cantidad de pallets enviadas desde cada PUERTO a cada VINEDO

In [136]:
modelo.x = Var(VINEDOS, PUERTOS, domain=NonNegativeReals) # Número de pallets enviados desde un viñedo hacia un puerto determinado

In [ ]:
# Función objetivo: Minimizar costo de transporte
# Minimizar la sumatoria del costo por pallet * número da pallets

In [138]:
modelo.CostoTransporte = Objective(
    expr=sum(Costos[v,p]*modelo.x[v,p] for v in VINEDOS for p in PUERTOS), sense=minimize)

In [ ]:
# Restricciones de oferta de los viñedos
# La sumatoria de los pallets enviadas de un viñedo respectivo a los puertos debe ser menor a la Oferta en pallets
# dicho viñedo
# .add() va añadiendo restricciones a la lista ConstraintList()

In [140]:
modelo.restriccionVinedo = ConstraintList()
for v in VINEDOS:
    modelo.restriccionVinedo.add(sum(modelo.x[v,p] for p in PUERTOS) <= Oferta[v])

In [ ]:
# Restricciones para satisfacer la demanda de cada puerto
# La sumatoria de los envíos de todos los viñedos debe ser igual a la demanda del puerto respectivo

In [141]:
modelo.restriccionPuerto = ConstraintList()
for p in PUERTOS:
    modelo.restriccionPuerto.add(sum(modelo.x[v,p] for v in VINEDOS) == Demanda[p])

In [ ]:
# Resolver el modelo

In [142]:
solucion = SolverFactory('glpk').solve(modelo)

In [ ]:
# Imprimir resultados

In [144]:
print("Costo Total de Transporte CLP:", value(modelo.CostoTransporte))
print("Con los siguientes envíos:")
for v in VINEDOS: 
    for p in PUERTOS:
        if value(modelo.x[v,p]) >0:
            print("Envíe desde el viñedo en", v, "al puerto", p, value(modelo.x[v,p]), "pallets" )

Costo Total de Transporte CLP: 23040000.0
Con los siguientes envíos:
Envíe desde el viñedo en Maipo al puerto Valparaiso 115.0 pallets
Envíe desde el viñedo en Maipo al puerto SanAntonio 125.0 pallets
Envíe desde el viñedo en Casablanca al puerto Valparaiso 360.0 pallets
Envíe desde el viñedo en Colchagua al puerto SanAntonio 385.0 pallets
